# Transfer Learning Attempts

## A. Fine-tuning CNN

In [5]:
#load images and labels
import pandas as pd
import numpy as np

movies = pd.read_csv('data/mov-cleaned.csv')
movies = movies.set_index("id")
rated_movies = movies[movies.vote_count != 0]
posters = rated_movies[['poster_path','vote_average']]

posters = posters.dropna(axis=0,how='any')
rated_movies

,adult,backdrop_path,belongs_to_collection,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,...,spoken_languages,status,tagline,title,vote_count,vote_average,movie_id,Actor1_pop,Actor2_pop,Actor3_pop
id,,,,,,,,,,,,,,,,,,,,,
339964,False,/o6OhxtsgMurL4h68Uqei0aSPMNr.jpg,NaN,"[{'name': 'Adventure', 'id': 12}, {'name': 'Sc...",tt2239822,en,Valerian and the City of a Thousand Planets,"In the 28th century, Valerian and Laureline ar...",278.579364,/jfIpMh79fGRqYJ6PwZLCntzgxlF.jpg,...,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Valerian and the City of a Thousand Planets,1347,6.7,339964,9.460488,5.426100,4.771382
35691,False,/r9OuqEdge1AXizB8JTcsMuUV17T.jpg,NaN,"[{'name': 'Action', 'id': 28}, {'name': 'Drama...",tt1462054,en,Letters to God,A young boy fighting cancer writes letters to ...,18.178478,/coraGbtZPMKUQEFRKN9Epnqi0UA.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Letters to God,39,6.6,35691,2.222239,0.860164,0.715197
112160,False,/dhsYvPtiBoNfxXqIsBKqV7cTQwm.jpg,NaN,"[{'name': 'Comedy', 'id': 35}, {'name': 'Drama...",tt1753584,fr,Chroniques sexuelles d'une famille d'aujourd'hui,Three generations of a French family open up a...,10.089318,/pU2nD3Llaz3Taq5FAaQUg00uJYD.jpg,...,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Sexual Chronicles of a French Family,96,4.2,112160,0.816190,0.615657,0.614648
64690,False,/ekLAySalu0CGoGogPWY0BG6uAul.jpg,NaN,"[{'name': 'Drama', 'id': 18}, {'name': 'Action...",tt0780504,en,Drive,A Hollywood stunt performer who moonlights as ...,12.873933,/nu7XIa67cXc2t7frXCE5voXUJcN.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are no clean getaways.,Drive,4118,7.5,64690,6.754838,6.189796,5.170999
1018,False,/ihkQywNs6Si24WssdmT1rNHuAYy.jpg,NaN,"[{'name': 'Thriller', 'id': 53}, {'name': 'Dra...",tt0166924,en,Mulholland Drive,After a car wreck on the winding Mulholland Dr...,10.699658,/oKyY4TFaLjQTgyX8oRde82GinOw.jpg,...,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,An actress longing to be a star. A woman searc...,Mulholland Drive,1655,7.7,1018,7.209498,3.459744,2.761127
10285,False,/rNqayZdhu5yCckGRjMsrpbsi55Z.jpg,1.0,"[{'name': 'Fantasy', 'id': 14}, {'name': 'Horr...",tt0107254,en,Jason Goes to Hell: The Final Friday,"Jason Voorhees, the living, breathing essence ...",9.189482,/8yLdCrD1Eh6eoGTIIQ4jNBFLkUb.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Evil has finally found a home.,Jason Goes to Hell: The Final Friday,207,4.2,10285,0.529973,0.412029,0.214305
73861,False,/19qAk3bQZ6ulspbwhPw1A0bNpii.jpg,NaN,"[{'name': 'Horror', 'id': 27}, {'name': 'Crime...",tt1273235,sr,Srpski film,"Milos, a retired porn star, leads a normal fam...",12.296960,/edM906NOM5WxaaAMeTsgDeqnBDr.jpg,...,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Not all films have a happy ending.,A Serbian Film,446,5.5,73861,1.884415,1.059850,0.917087
298312,False,/cuAeZUGDQavjD0Ylc2g0ZVh401s.jpg,NaN,"[{'name': 'Horror', 'id': 27}, {'name': 'Thril...",tt3567288,en,The Visit,The terrifying story of a brother and sister w...,8.234061,/32tNtTEQ1vIOLm1hEOSFkVwJT5D.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one loves you like your grandparents.,The Visit,1524,6.1,298312,2.838991,1.722327,1.716000
8967,False,/tBs9alJ2weUkOW83RkuBlz8Nlw6.jpg,NaN,"[{'name': 'Drama', 'id': 18}, {'name': 'Fantas...",tt0478304,en,The Tree of Life,The impressionistic story of a Texas family in...,11.621256,/ptDOdfOg0srtk4TGdeYbLqxv2nd.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Nothing stands still.,The Tree of Life,1080,6.5,8967,10.251028,5.901165,2.526580


In [6]:
from keras.preprocessing import image
path = 'data/small/poster'
X = []
y = []
for i in posters.values:
    try:
        img = image.load_img(path + i[0])
        X.append(image.img_to_array(img))
        y.append(i[1])
    except:
        print(i)
X = np.array(X)
y = np.array(y)

X.shape

(2741, 114, 92, 3)

In [7]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

def base_model():
    #Get back the convolutional part of a VGG network trained on ImageNet
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
    model_vgg16_conv.summary()

    for l in model_vgg16_conv.layers[0:15]:
        l.trainable = False

    #Create your own input format (here 3x200x200)
    input = Input(shape=(114,92,3),name = 'image_input')

    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(input)

    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1, activation='softmax', name='predictions')(x)
    
    #Create your own model 
    my_model = Model(input=input, output=x)
    
    my_model.compile(loss="mean_squared_error", optimizer="adam")
    

    return my_model


# #In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
# my_model.summary()

# #Then training with your data ! 

In [8]:
clf = KerasRegressor(build_fn=base_model, nb_epoch=100, batch_size=5,verbose=1)

In [9]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2,random_state=0)

In [10]:
clf.fit(train_X, train_y,validation_data=(test_X,test_y),verbose=1,epochs=10,batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`


Train on 2192 samples, validate on 549 samples
Epoch 1/10
2192/2192 [==============================] - 20s - loss: 24.3742 - val_loss: 23.2452
Epoch 2/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 3/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 4/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 5/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 6/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 7/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 8/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 9/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 23.2452
Epoch 10/10
2192/2192 [==============================] - 18s - loss: 24.3742 - val_loss: 

In [14]:
features = clf.predict(test_X, batch_size=5, verbose=1)
print(features)

549/549 [==============================] - 3s     
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.


## B. Fixed Feature Extraction using VGG 16

In [15]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

# use VGG16 model extract feature from fc1 layer
base_model = VGG16(weights='imagenet', pooling=max, include_top = False)
input = Input(shape=(114,92,3),name = 'image_input')
x = base_model(input)
x = Flatten()(x)
model = Model(inputs=input, outputs=x)

In [16]:
from keras.preprocessing import image
train_path = 'data/small/poster'
X_train = []
y_train = []
for f, tags in posters.values[:]:
    img_path = train_path + f
    img = image.load_img(img_path, target_size=(114,92))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    features_reduce =  features.squeeze()
    X_train.append(features_reduce)


In [20]:
train_features =  pd.DataFrame(np.array(X_train))
train_features.to_csv('train_feature.csv')